<a href="https://colab.research.google.com/github/DmitryKutsev/congrats_generator/blob/feature%234/my_gpt3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://colab.research.google.com/github/sberbank-ai/ru-gpts/blob/master/examples/Finetune_RuGPTs_with_HF.ipynb#scrollTo=aZ-4Kav28cH0




# **IMPORT**

In [46]:
import pandas as pd

In [47]:
!pip install transformers -U

In [48]:
!pip install sentencepiece

In [49]:
!pip install tqdm boto3 requests regex sacremoses

In [50]:
import torch
from transformers import AutoModel, AutoTokenizer, AutoConfig

In [51]:
# model = torch.hub.load('huggingface/transformers', 'modelForCausalLM', 'gpt2')   
#  # Download model and configuration from huggingface.co and cache.
# # model = torch.hub.load('huggingface/transformers', 'modelForCausalLM', './test/saved_model/')  
# # E.g. model was saved using `save_pretrained('./test/saved_model/')`
# model = torch.hub.load('huggingface/transformers', 'modelForCausalLM', 'gpt2', output_attentions=True)  
# # Update configuration during loading
# assert model.config.output_attentions == True
# # Loading from a TF checkpoint file instead of a PyTorch model (slower)
# config = AutoConfig.from_pretrained('bert-base-uncased')
# # AutoConfig.from_pretrained('bert-base-uncased')
# # model = torch.hub.load('huggingface/transformers', 'modelForCausalLM', 
# #                        './tf_model/gpt_tf_checkpoint.ckpt.index', from_tf=True, config=config)

In [52]:
!git clone https://github.com/huggingface/transformers.git

fatal: destination path 'transformers' already exists and is not an empty directory.


In [53]:
!pip install tensorboard
# !pip install wandb; wandb login

In [54]:
!pip install natasha

In [55]:
from natasha import (   
    MorphVocab,
    NewsNERTagger,
    NamesExtractor,
    MorphVocab,
    Segmenter, 
    NewsEmbedding,
    PER,
    Doc,
    ORG
)

In [56]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [57]:
!pip install awscli
!aws s3 sync --no-sign-request s3://models.dobro.ai/gpt2/ru/unfreeze_all gpt2

# **PREPARE DATA SET**

In [58]:
!wget https://raw.githubusercontent.com/DmitryKutsev/congrats_generator/main/crawler/curr_congrats.txt

--2021-12-24 16:46:58--  https://raw.githubusercontent.com/DmitryKutsev/congrats_generator/main/crawler/curr_congrats.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1471875 (1.4M) [text/plain]
Saving to: ‘curr_congrats.txt.1’

curr_congrats.txt.1 100%[===================>]   1.40M  --.-KB/s    in 0.02s   

2021-12-24 16:46:58 (65.1 MB/s) - ‘curr_congrats.txt.1’ saved [1471875/1471875]



In [59]:
with open('curr_congrats.txt') as handler:
  full_text = handler.read()
full_text[:3000]

'\nTITLE\nЭдварду Радзинскому, писателю, драматургу, сценаристу\nCONTENT\n Уважаемый Эдвард Станиславович!Примите поздравления с\xa085-летием.Человек многогранного дарования и\xa0большой созидательной энергии, Вы по\xa0праву входите в\xa0число признанных\nдраматургов, мастеров современного театрального искусства. И\xa0конечно, Вас\nхорошо знают и\xa0любят как автора целого ряда просветительских проектов, чьё\nтворчество завоевало сердца самой широкой аудитории, в\xa0том числе молодёжи.Желаю доброго здоровья, благополучия и\xa0бодрости\nдуха. Владимир Путин     Статус материала Опубликован в\xa0разделе: Телеграммы  Ссылка на материал: kremlin.ru/d/66751   Текстовая версия      \nEND\n\nTITLE\nКоллективу радиостанции «Орфей»\nCONTENT\n Дорогие друзья!Поздравляю вас со\xa0знаменательным юбилеем\xa0– 30-летием радиостанции «Орфей».Все эти годы ваш коллектив объединяет настоящих единомышленников, профессионалов, преданных высокому искусству, стремящихся внести свой вклад в\xa0сбережение луч

In [60]:
full_text.split('END')[:3][0].replace('\n', '').split('CONTENT')

['TITLEЭдварду Радзинскому, писателю, драматургу, сценаристу',
 ' Уважаемый Эдвард Станиславович!Примите поздравления с\xa085-летием.Человек многогранного дарования и\xa0большой созидательной энергии, Вы по\xa0праву входите в\xa0число признанныхдраматургов, мастеров современного театрального искусства. И\xa0конечно, Васхорошо знают и\xa0любят как автора целого ряда просветительских проектов, чьётворчество завоевало сердца самой широкой аудитории, в\xa0том числе молодёжи.Желаю доброго здоровья, благополучия и\xa0бодростидуха. Владимир Путин     Статус материала Опубликован в\xa0разделе: Телеграммы  Ссылка на материал: kremlin.ru/d/66751   Текстовая версия      ']

In [61]:
titles_list = []
congrats_list = []

for letter in full_text.split('END'):
  letter = letter.replace('\n', '').replace('TITLE', '')

  if len(letter.split('CONTENT')) > 1:
    titles_list.append(letter.split('CONTENT')[0])
    congrats_list.append(letter.split('CONTENT')[1])

print(titles_list[:3])
print(congrats_list[:3])
print(len(titles_list))
print(len(congrats_list))

['Эдварду Радзинскому, писателю, драматургу, сценаристу', 'Коллективу радиостанции «Орфей»', 'Михаилу Ковальчуку, президенту НИЦ «Курчатовский институт»']
[' Уважаемый Эдвард Станиславович!Примите поздравления с\xa085-летием.Человек многогранного дарования и\xa0большой созидательной энергии, Вы по\xa0праву входите в\xa0число признанныхдраматургов, мастеров современного театрального искусства. И\xa0конечно, Васхорошо знают и\xa0любят как автора целого ряда просветительских проектов, чьётворчество завоевало сердца самой широкой аудитории, в\xa0том числе молодёжи.Желаю доброго здоровья, благополучия и\xa0бодростидуха. Владимир Путин     Статус материала Опубликован в\xa0разделе: Телеграммы  Ссылка на материал: kremlin.ru/d/66751   Текстовая версия      ', ' Дорогие друзья!Поздравляю вас со\xa0знаменательным юбилеем\xa0– 30-летием радиостанции «Орфей».Все эти годы ваш коллектив объединяет настоящих единомышленников, профессионалов, преданных высокому искусству, стремящихся внести свой вкла

In [62]:
data = {'Title': titles_list,
        'Content': congrats_list}

my_df = pd.DataFrame(data)
my_df

,Title,Content
0,"Эдварду Радзинскому, писателю, драматургу, сце...",Уважаемый Эдвард Станиславович!Примите поздра...
1,Коллективу радиостанции «Орфей»,Дорогие друзья!Поздравляю вас со знаменательн...
2,"Михаилу Ковальчуку, президенту НИЦ «Курчатовск...",Уважаемый Михаил Валентинович!Примите мои поз...
3,"Ирине Андреевой и Ивану Штылю, победителям чем...",Уважаемые Ирина Александровна и Иван Александ...
4,Работникам и ветеранам оборонно-промышленного ...,Уважаемые друзья!Поздравляю вас с Днём оружей...
...,...,...
883,Жителям Республики Мордовия,Дорогие друзья!Поздравляю вас с юбилеем – 75-...
884,Коллективу Института физических проблем им.П.Л...,Поздравляю коллектив института с 70-летием со...
885,Жителям Красноярского края,Уважаемые друзья!Поздравляю вас с 70-летием с...
886,В.А.ЛОГИНОВУ,Уважаемый Владимир Александрович!Поздравляю В...


In [63]:
!pip install datasets transformers

In [64]:
!git clone  https://github.com/sberbank-ai/ru-gpts

fatal: destination path 'ru-gpts' already exists and is not an empty directory.


In [65]:
from datasets import load_dataset
from sklearn.model_selection import train_test_split
import re


In [66]:
my_df['Sum'] = [my_df['Title'].tolist()[i] + my_df['Content'].tolist()[i] for i in range(len(my_df))]
my_df

,Title,Content,Sum
0,"Эдварду Радзинскому, писателю, драматургу, сце...",Уважаемый Эдвард Станиславович!Примите поздра...,"Эдварду Радзинскому, писателю, драматургу, сце..."
1,Коллективу радиостанции «Орфей»,Дорогие друзья!Поздравляю вас со знаменательн...,Коллективу радиостанции «Орфей» Дорогие друзья...
2,"Михаилу Ковальчуку, президенту НИЦ «Курчатовск...",Уважаемый Михаил Валентинович!Примите мои поз...,"Михаилу Ковальчуку, президенту НИЦ «Курчатовск..."
3,"Ирине Андреевой и Ивану Штылю, победителям чем...",Уважаемые Ирина Александровна и Иван Александ...,"Ирине Андреевой и Ивану Штылю, победителям чем..."
4,Работникам и ветеранам оборонно-промышленного ...,Уважаемые друзья!Поздравляю вас с Днём оружей...,Работникам и ветеранам оборонно-промышленного ...
...,...,...,...
883,Жителям Республики Мордовия,Дорогие друзья!Поздравляю вас с юбилеем – 75-...,Жителям Республики Мордовия Дорогие друзья!Поз...
884,Коллективу Института физических проблем им.П.Л...,Поздравляю коллектив института с 70-летием со...,Коллективу Института физических проблем им.П.Л...
885,Жителям Красноярского края,Уважаемые друзья!Поздравляю вас с 70-летием с...,Жителям Красноярского края Уважаемые друзья!По...
886,В.А.ЛОГИНОВУ,Уважаемый Владимир Александрович!Поздравляю В...,В.А.ЛОГИНОВУ Уважаемый Владимир Александрович!...


# **NER**

In [67]:
emb = NewsEmbedding()
morph_vocab = MorphVocab()

names_extractor = NamesExtractor(morph_vocab)
ner_tagger = NewsNERTagger(emb)

In [68]:
text = "Эдварду Радзинскому, писателю, драматургу, сценаристуКоллективу радиостанции «Орфей», Михаилу Ковальчуку, президенту НИЦ «Курчатовский институт»"
doc = Doc(text)

In [69]:
doc

Doc(text='Эдварду Радзинскому, писателю, драматургу, сценар...)

In [70]:
segmenter = Segmenter()
doc.segment(segmenter)

In [71]:
doc.tag_ner(ner_tagger)

In [72]:
for span in doc.spans:
    print(span)
    # if span.type == PER:
    #   print(span)


DocSpan(stop=19, type='PER', text='Эдварду Радзинскому', tokens=[...])
DocSpan(start=78, stop=83, type='ORG', text='Орфей', tokens=[...])
DocSpan(start=86, stop=104, type='PER', text='Михаилу Ковальчуку', tokens=[...])
DocSpan(start=117, stop=144, type='ORG', text='НИЦ «Курчатовский институт»', tokens=[...])


***Можно маскировать как имена, так и оргинизации. Попробую для начала и то, и другое вместе.***

In [73]:
masked_content = []

for sent in my_df['Content']:
  doc = Doc(sent)
  doc.segment(segmenter)
  doc.tag_ner(ner_tagger)

  for span in doc.spans:
      if span.type == PER or span.type == ORG:
          sent = sent.replace(span.text, 'MASK')

  masked_content.append(sent)


***Title оставляем прежним, маскируем контент. Модели будет скармливаться колонка с конкатенированным текстом.***

In [74]:
my_df['Masked_content'] = masked_content
my_df['Sum'] = [my_df['Title'].tolist()[i] + my_df['Masked_content'].tolist()[i] for i in range(len(my_df))]
my_df['Sum'].to_list()[:3]

['Эдварду Радзинскому, писателю, драматургу, сценаристу Уважаемый MASK!Примите поздравления с\xa085-летием.Человек многогранного дарования и\xa0большой созидательной энергии, Вы по\xa0праву входите в\xa0число признанныхдраматургов, мастеров современного театрального искусства. И\xa0конечно, MASK знают и\xa0любят как автора целого ряда просветительских проектов, чьётворчество завоевало сердца самой широкой аудитории, в\xa0том числе молодёжи.Желаю доброго здоровья, благополучия и\xa0бодростидуха. MASK     Статус материала Опубликован в\xa0разделе: Телеграммы  Ссылка на материал: kremlin.ru/d/66751   Текстовая версия      ',
 'Коллективу радиостанции «Орфей» Дорогие друзья!Поздравляю вас со\xa0знаменательным юбилеем\xa0– 30-летием радиостанции «MASK».Все эти годы ваш коллектив объединяет настоящих единомышленников, профессионалов, преданных высокому искусству, стремящихся внести свой вклад в\xa0сбережение лучших традиций отечественной культуры. Вы сформировали обширный фонд просветительск

# **PREPARE DATA FOR TRAINING**

In [75]:
train_test_ratio = 0.9
df_train, df_test = train_test_split(my_df, train_size = train_test_ratio, random_state = 1)
# train_valid_ratio = 7/9
# df_train, df_valid = train_test_split(df_full_train, train_size = train_valid_ratio, random_state = 1)

***Первичная очистка данных от мусора типа "Ссылка на метериал"***

In [76]:
def build_dataset(df, dest_path):
    f = open(dest_path, 'w')
    data = ''
    summaries = df['Sum'].tolist()

    for summary in summaries:
        summary = str(summary).strip()
        summary = re.sub(r"\s", " ", summary)
        summary = re.sub(r"\n", "", summary)
        summary = re.sub(r"Статус материала Опубликован в разделе: Телеграммы", "", summary)
        summary = re.sub(r"Ссылка на материал:", "", summary)
        summary = re.sub(r"Текстовая версия:", "", summary)
        summary = re.sub(r"Ссылка на материал:", "", summary)
        summary = re.sub(r"MASK", "МАСК", summary)
        summary = re.sub(r"([a-zA-Z]+)", "", summary)
        summary = re.sub(r"МАСК", "MASK", summary)
        summary = re.sub(r"([0-9/*#@+]+)", "", summary)
        data += summary

    f.write(data)

In [32]:
!ls
!rm 'train.txt' 'test.txt'
!ls

curr_congrats.txt  gpt2  ru-gpts  sample_data  transformers
rm: cannot remove 'train.txt': No such file or directory
rm: cannot remove 'test.txt': No such file or directory
curr_congrats.txt  gpt2  ru-gpts  sample_data  transformers


In [77]:
build_dataset(df_train, 'train.txt')
build_dataset(df_test, 'test.txt')
# build_dataset(df_valid, 'valid.txt')

***Проверка тескста на наличие MASK***

In [79]:
with open('train.txt') as handler:
  rrr = handler.read()
rrr[:1010]

'Коллективу «Российской газеты» Дорогие друзья!Поздравляю вас со знаменательной датой – -летиемсоздания «MASK».За прошедшие годы газета стала одним из ведущихпериодических изданий страны, завоевала уважение большой читательской аудиториии авторитет в отечественном медиасообществе. Её ценят за информационнуюнаполненность, глубокий подход к освещению важнейших тем и событий, за взвешенность и основательность оценок. Отрадно, что все эти годы ваш коллективвысоко держит планку качества работы, сохраняет приверженность лучшим традициямжурналистики, расширяет линейку информационных продуктов – от публикациигосударственных документов до новостей, репортажей и серьёзной аналитики.Уверен, что вы и впредь будете добросовестнотрудиться, воплощать в жизнь востребованные творческие проекты, добиватьсяуспеха в профессиональной деятельности.Желаю вам всего наилучшего. MASK     Темы  MASK           .   Текстовая версияРаботникам и ветеранам автомобильного и городского пассажирского транспорта России У

In [34]:
# tokenizer = AutoTokenizer.from_pretrained("anonymous-german-nlp/german-gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")
 
train_path = 'train.txt'
test_path = 'test.txt'

Downloading:   0%|          | 0.00/1.63M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/608 [00:00<?, ?B/s]

In [35]:
!rm ./gpt-letters

rm: cannot remove './gpt-letters': No such file or directory


In [36]:
from transformers import TextDataset, DataCollatorForLanguageModeling

def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)
     
    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset, test_dataset, data_collator

train_dataset, test_dataset, data_collator = load_dataset(train_path, test_path, tokenizer)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


## **TRAIN MODEL**

In [37]:
from transformers import Trainer, TrainingArguments, AutoModelWithLMHead

model = GPT2LMHeadModel.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")


training_args = TrainingArguments(
    output_dir="./gpt-letters", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    # num_train_epochs=3, # number of training epochs
    # per_device_train_batch_size=32, # batch size for training
    # per_device_eval_batch_size=64,  # batch size for evaluation
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=16, # batch size for training
    per_device_eval_batch_size=16,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=150, # after # steps model is saved 
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

Downloading:   0%|          | 0.00/526M [00:00<?, ?B/s]

In [38]:
trainer.train()

***** Running training *****
  Num examples = 936
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 177


Step,Training Loss


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-38-3435b262f1ae>", line 1, in <module>
    trainer.train()
  File "/usr/local/lib/python3.7/dist-packages/transformers/trainer.py", line 1332, in train
    tr_loss_step = self.training_step(model, inputs)
  File "/usr/local/lib/python3.7/dist-packages/transformers/trainer.py", line 1909, in training_step
    loss.backward()
  File "/usr/local/lib/python3.7/dist-packages/torch/_tensor.py", line 307, in backward
    torch.autograd.backward(self, gradient, retain_graph, create_graph, inputs=inputs)
  File "/usr/local/lib/python3.7/dist-packages/torch/autograd/__init__.py", line 156, in backward
    allow_unreachable=True, accumulate_grad=True)  # allow_unreachable flag
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most rece

KeyboardInterrupt: ignored

In [39]:
trainer.save_model()

Saving model checkpoint to ./gpt-letters
Configuration saved in ./gpt-letters/config.json
Model weights saved in ./gpt-letters/pytorch_model.bin


In [41]:
# my_model = torch.load('./gpt-letters/pytorch_model.bin')
from transformers import pipeline

pu = pipeline('text-generation',model='./gpt-letters', tokenizer=tokenizer, max_length=500)

loading configuration file ./gpt-letters/config.json
Model config GPT2Config {
  "_name_or_path": "./gpt-letters",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 2048,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 2048,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "torch_dtype": "float32",
  "transformers_version": "4.15.0",
  "use_cache": true,
  "vocab_size": 50264
}

loading configuration file ./gpt-letters/config.json
Model config GPT2Config {
  "_

In [42]:
pu("Александру Сергеевичу Пушкину")[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Александру Сергеевичу Пушкину, известному российскому поэту, писателю, драматургу, режиссёру, лауреату Государственной премии СССР, почётному гражданину города Москвы Уважаемый! Примите мои поздравления с -летием и наилучшие пожелания. Желаю Вам здоровья, благополучия и всего самого доброго.        .   Текстовая версияВалерию Яковлеву, поэту, драматургу, лауреату Государственной премии СССР Уважаемый! Примите мои поздравления с -летием и наилучшие пожелания. Вы - выдающийся поэт, драматург, автор замечательных произведений, в которых воплощены лучшие традиции русской литературы. Вы - яркий, самобытный, самобытный поэт, автор замечательных произведений, в которых воплощены лучшие традиции русской литературы. Вы - яркий, самобытный, самобытный поэт, автор замечательных произведений, в которых воплощены лучшие традиции русской литературы. Вы - яркий, самобытный, самобытный поэт, автор замечательных произведений, в которых воплощены лучшие традиции русской литературы. Вы - яркий, самобытн

In [43]:
pu("Кровавым убийцам детей")[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Кровавым убийцам детей и молодёжи, которые в годы Великой Отечественной войны были активными участниками боевых действий на территории нашей страны, а также в годы Великой Отечественной войны, в годы Великой Отечественной войны, в годы Великой Отечественной войны, в годы Великой Отечественной войны, в годы Великой Отечественной войны, в годы Великой Отечественной войны, в годы Великой Отечественной войны, в годы Великой Отечественной войны, в годы Великой Отечественной войны, в годы Великой Отечественной войны, в годы Великой Отечественной войны, в годы Великой Отечественной войны, в годы Великой Отечественной войны, в годы Великой Отечественной войны, в годы Великой Отечественной войны, в годы Великой Отечественной войны, в годы Великой Отечественной войны, в годы Великой Отечественной войны, в годы Великой Отечественной войны, в годы Великой Отечественной войны, в годы Великой Отечественной войны, в годы Великой Отечественной войны, в годы Великой Отечественной войны, в годы Великой

In [44]:
pu("Уважаемые осквернители святынь и насильники")[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Уважаемые осквернители святынь и насильники, осквернители святынь и насильники!Поздравляю вас с Днём рождения.Вы - один из самых ярких и самобытных представителей русской культуры, самобытного и самобытного народа. Вы - символ России, символ единства и согласия, символ единства и согласия. Вы - символ единства и согласия, символ единства и согласия. Вы - символ единства и согласия, символ единства и согласия. Вы - символ единства и согласия, символ единства и согласия. Вы - символ единства и согласия, символ единства и согласия. Вы - символ единства и согласия, символ единства и согласия. Вы - символ единства и согласия, символ единства и согласия. Вы - символ единства и согласия, символ единства и согласия. Вы - символ единства и согласия, символ единства и согласия. Вы - символ единства и согласия, символ единства и согласия. Вы - символ единства и согласия, символ единства и согласия. Вы - символ единства и согласия, символ единства и согласия. Вы - символ единства и согласия, симв

In [45]:
pu("Хочу сказать этому Обэме")[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


KeyboardInterrupt: ignored

In [ ]:
pu("Куцеву Диме")[0]['generated_text']

In [ ]:
pu("Мычалкин")[0]['generated_text']

In [ ]:
from google.colab import files

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!ls
!cp gpt-letters/pytorch_model.bin  gdrive/MyDrive/pytorch_model.bin

In [ ]:
import os
for file in os.listdir('./gpt-letters'):
  if str(file) != 'pytorch_model.bin':
    files.download(f'./gpt-letters/{file}')

In [ ]:
import os
for file in os.listdir('./gpt-letters'):
  print(file)

In [ ]:
!ls
!cp gpt-letters/ gdrive/MyDrive/gpt-letters